In [1]:
#
# David Laziuk
# Logistic Regression from scratch
#

# Two Methods Will Be Implemented:
## 1. Stochastic Gradient Descent
## 2. Analytical (Moore-Penrose pseudo inverse)

In [3]:
#Imports
import random
import numpy as np
import pandas as pd
#Loading Data
data=pd.read_csv('breast-cancer-wisconsin.csv',na_values= "?")
#Dropping rows with NAs
data=data.dropna()
#Fixing value type
data['F6']=data['F6'].astype(int)
#Mapping class(2/4) to (-1/1)
data['Class']=data['Class'].replace(2,-1)
data['Class']=data['Class'].replace(4,1)
data['Class']=data['Class'].astype(int)
#Dropping Sample Column
try: data=data.drop('Sample',axis=1) 
except KeyError: pass
#Standardizing (Z-Score Normalization)
feats=data.columns[:-1]
mean=data[feats].mean()
std=data[feats].std()
norm=(data[feats]-mean)/std
data[feats]=norm
#Shuffling Entries
data=data.sample(frac=1).reset_index(drop=True)
#70/30 split into train/test data
train_size=int(np.floor(data.shape[0]*.7))
features=(data.shape[1]-1)
X_train=data.iloc[0:train_size,0:features].reset_index(drop=True).values
X_test=data.iloc[train_size:,0:features].reset_index(drop=True).values
y_train=data.iloc[0:train_size,features].reset_index(drop=True).values
y_test=data.iloc[train_size:,features].reset_index(drop=True).values
#Appending Bias Column
X_train=np.concatenate((X_train,np.ones((X_train.shape[0],1))),axis=1)
X_test=np.concatenate((X_test,np.ones((X_test.shape[0],1))),axis=1)
del features,train_size,mean,std,feats,norm

# 1. Stochastic Gradient Descent:

In [4]:
#SGD
def grad(w,xi,yi):
    yx=np.multiply(yi,xi)
    yx=np.reshape(yx,(1,xi.shape[0]))
    yxw=np.dot(yx,w)
    grad=((-yx.T)/(1+np.exp(yxw)))
    return grad

def sgd(X_train,y_train,epochs,lr,X_test):
    #Initializing Weights
    w=np.zeros((X_train.shape[1],1))
    #Iterating through epochs
    for e in range(epochs):
        #Randomly shuffling data every epoch
        r=np.random.permutation(X_train.shape[0])
        X_temp=X_train[r,:]
        y_temp=y_train[r]
        #Iterating through samples
        for i in range(X_train.shape[0]):
            #Extracting individual sample
            xi=X_temp[i,:]
            yi=y_temp[i]
            g=grad(w,xi,yi)
            w-=(lr*g)
    #Finding Predictions From Learned Weight
    z=np.dot(X_test,w)
    #Sigmoid activation
    sig=1/(1+np.exp(-z))
    #Thresholding
    y_pred=np.zeros((X_test.shape[0]))
    for i in range(sig.shape[0]):
        y_pred[i]=1 if(sig[i]>=.5) else -1
    return y_pred

#Adding Metrics Function
def test(yp,yt):
    fp,fn,tp,tn=0,0,0,0
    for i in range(yp.shape[0]):
        if(yt[i]==1 and yp[i]==1): tp+=1 
        if(yt[i]==-1 and yp[i]==-1): tn+=1 
        if(yt[i]==-1 and yp[i]==1): fp+=1 
        if(yt[i]==1 and yp[i]==-1): fn+=1 
    acc=(tp+tn)/yp.shape[0]
    #Precision, Recall, F1 can be added
    return acc

In [5]:
#Testing SGD
epochs=100
learning_rate=.1
y_pred=sgd(X_train,y_train,epochs,learning_rate,X_test)
acc=test(y_pred,y_test)
print("Accuracy: "+str(acc))

Accuracy: 0.9853658536585366


# 2. Analytical (Moore-Penrose pseudo inverse)

In [6]:
# Pseudo inverse
X_pinv=np.linalg.pinv(X_train)
w=np.dot(X_pinv,y_train)
#Finding Predictions From Learned Weight
z=np.dot(X_test,w)
#Sigmoid activation
sig=1/(1+np.exp(-z))
#Thresholding
y_pred=np.zeros((X_test.shape[0]))
for i in range(sig.shape[0]):
    y_pred[i]=1 if(sig[i]>=.5) else -1
#Testing
acc=test(y_pred,y_test)
print("Accuracy: "+str(acc))

Accuracy: 0.975609756097561
